# Stepwise Regression

From [STAT 501](https://newonlinecourses.science.psu.edu/stat501/node/329/)

In this section, we learn about the stepwise regression procedure. While we will soon learn the finer details, the general idea behind the stepwise regression procedure is that we build our regression model from a set of candidate predictor variables by entering and removing predictors — in a stepwise manner — into our model until there is no justifiable reason to enter or remove any more


## An example

Let's learn how the stepwise regression procedure works by considering a data set that concerns the hardening of cement. 

In particular, the researchers were interested in learning how the composition of the cement affected the heat evolved during the hardening of the cement

Therefore, they measured and recorded the following data on 13 batches of cement:

- Response y: heat evolved in calories during hardening of cement on a per gram basis
- Predictor x1: % of tricalcium aluminate
- Predictor x2: % of tricalcium silicate
- Predictor x3: % of tetracalcium alumino ferrite
- Predictor x4: % of dicalcium silicate

### Upload data

In [6]:
from GoogleDrivePy.google_authentification import connect_service_local
from GoogleDrivePy.google_drive import connect_drive
import pandas as pd
import numpy as np
import os
import re
import statsmodels.api as sm
pathcredential = '/Users/Thomas/Google Drive/Projects/Data_science/Google_code_n_Oauth/Client_Oauth/Google_auth/'
scopes = ['https://www.googleapis.com/auth/documents.readonly',
            'https://www.googleapis.com/auth/drive', 
         'https://www.googleapis.com/auth/spreadsheets.readonly']
serviceaccount = '/Users/Thomas/Google Drive/Projects/Data_science/Google_code_n_Oauth/Client_Oauth/Google_auth/valid-pagoda-132423-c6ac84b41833.json'
cs = connect_service_local.connect_service_local(path_json =pathcredential,
                                                 path_service_account = serviceaccount,
                                                 scope = scopes)
service = cs.get_service()



Service Google Drive and Docs, Sheet are now connected. 
Service Google Drive is stored as <googleapiclient.discovery.Resource object at 0x1a1ab56860> and accessible with "drive" 
Service Google Doc is stored as <googleapiclient.discovery.Resource object at 0x1c1fb5f1d0> and accessible with "doc" 
Service Google Sheet is stored as <googleapiclient.discovery.Resource object at 0x1c1fc40dd8>and accessible with "sheet"


In [0]:
headers = ['y','x1','x2','x3','x4']
range_name = 'cement.csv!A2:E14'

df = service['sheet'].spreadsheets().values().get(
    spreadsheetId='1QvLOtbVBOaeDpDzg5gtrarIlcKgorC1qgVbqpNyIlCQ',
    range=range_name).execute()
df = pd.DataFrame(df.get('values', []), columns=headers)
df = df.apply(pd.to_numeric, errors='ignore')

### Scatter plot matrix

you can get a hunch of which predictors are good candidates for being the first to enter the stepwise model. 

It looks as if the strongest relationship exists between either y and x2 or between y and x4 — and therefore, perhaps either x2 or x4 should enter the stepwise model first.

Did you notice what else is going on in this data set though? A strong correlation also exists between the predictors x2 and x4! 

How does this correlation among the predictor variables play out in the stepwise procedure? Let's see what happens when we use the stepwise regression method to find a model that is appropriate for these data

**Note**. The number of predictors in this data set is not large. The stepwise procedure is typically used on much larger data sets, for which it is not feasible to attempt to fit all of the possible regression models. For the sake of illustration, the data set here is necessarily small, so that the largeness of the data set does not obscure the pedagogical point being made.

In [5]:
import seaborn as sns
sns.set(style="ticks")

sns.pairplot(df)

## The procedure

First, we start with no predictors in our "stepwise model." Then, at each step along the way we either enter or remove a predictor based on the partial F-tests — that is, the t-tests for the slope parameters — that are obtained. 

We stop when no more predictors can be justifiably entered or removed from our stepwise model, thereby leading us to a "final model."

Starting the procedure. The first thing we need to do is set a significance level for deciding when to enter a predictor into the stepwise model. 

We'll call this the Alpha-to-Enter significance level and will denote it as $\alpha_{\mathrm{E}}$. Of course, we also need to set a significance level for deciding when to remove a predictor from the stepwise model. 

We'll call this the Alpha-to-Remove significance level and will denote it as $\alpha_{\mathrm{R}}$.

- Specify an Alpha-to-Enter significance level. This will typically be greater than the usual 0.05 level so that it is not too difficult to enter predictors into the model. 
- Specify an Alpha-to-Remove significance level. This will typically be greater than the usual 0.05 level so that it is not too easy to remove predictors from the model.



### Step 1

Once we've specified the starting significance levels, then we:

1. Fit each of the one-predictor models — that is, regress y on x1, regress y on x2, ..., and regress y on xp-1.
2. Of those predictors whose t-test P-value is less than αE = 0.15, the first predictor put in the stepwise model is the predictor that has the smallest t-test P-value.
3. If no predictor has a t-test P-value less than $\alpha_{\mathrm{E}} = 0.15$, stop.

In [0]:
import statsmodels.api as sm

list_x = ['x1','x2','x3','x4']

In [0]:
y = df['y'].to_numpy()
dic_stepwise = {
    'var': [],
    'p_value': []
}
#dic_stepwise['var'].update('X1')

In [0]:
p_value_final = 0.15
var_final = ''
for var in list_x:
  x = df[var].to_numpy()
  x = sm.add_constant(x)
  model = sm.OLS(y,x).fit()  
  p_value = model.pvalues[1]
  print("p_value: %f" % (p_value))
  if p_value < p_value_final:
    p_value_final = p_value
    var_final =  var

dic_stepwise['var'].append(var_final)
dic_stepwise['p_value'].append(p_value_final)
dic_stepwise

p_value: 0.004552
p_value: 0.000665
p_value: 0.059762
p_value: 0.000576


{'var': ['x4'], 'p_value': [0.0005762318164885003]}

### Step 2

1.  $x4$ has the smallest t-test P-value below $\alpha_{\mathrm{E}}=0.15$ and therefore was deemed the "best" single predictor arising from the the first step.
2. Now, fit each of the two-predictor models that include $x4$ as a predictor — that is, regress $y$ on $x4$ and $x2$, regress $y$ on $x4$ and $x3$, ..., and regress $y$ on$x1$ and $xp-1$.
3. Of those predictors whose t-test P-value is less than $\alpha_{\mathrm{E}}=0.15$, the second predictor put in the stepwise model is the predictor that has the smallest t-test P-value
4. If no predictor has a t-test P-value less than $\alpha_{\mathrm{E}}=0.15$, stop. The model with the one predictor obtained from the first step is your final model.

In [0]:
### Forward
p_value_final = 0.15
var_final = ''

for var in list_x:
  if var not in dic_stepwise['var']:
    
    list_temps = []
    for value in dic_stepwise['var']:
      list_temps.append(value)
    list_temps.append(var)
    
    x = df[list_temps].to_numpy()
    x = sm.add_constant(x)
    model = sm.OLS(y,x).fit()
    p_value = model.pvalues[2]
    
    print("p_value:" +  str(p_value))
    
    
    if p_value < p_value_final:
      p_value_final = p_value
      var_final =  var
print(var_final, p_value_final)

p_value:1.1052814195373175e-06
p_value:0.6866842279633266
p_value:8.375467286314457e-05
x1 1.1052814195373175e-06


5. But, suppose instead that x1 was deemed the "best" second predictor and it is therefore entered into the stepwise model.
6. Now, since $x4$ was the first predictor in the model, step back and see if entering $x1$ into the stepwise model somehow affected the significance of the $x4$ predictor. That is, check the t-test P-value for testing $\beta_{1}=0$. If the t-test P-value for\beta_{1}=0 has become not significant — that is, the P-value is greater than $\alpha_{\mathrm{R}}=0.15$ — remove $x4$ from the stepwise model.

In [0]:
  ### backward
if var_final != '':
  
  list_temps = []
  for value in dic_stepwise['var']:
    list_temps.append(value)
  list_temps.append(var_final)
    
    
  x = df[list_temps].to_numpy()
  x = sm.add_constant(x)
  model = sm.OLS(y,x).fit()
  p_value = model.pvalues[1]
  print("p_value:" +  str(p_value))
  
  
  if p_value < 0.15:
    dic_stepwise['var'].append(var_final)
    dic_stepwise['p_value'].append(p_value_final)
  else:
    dic_stepwise['var'][0] = var_final
    dic_stepwise['p_value'][1] = p_value_final
dic_stepwise 

p_value:1.814890465259972e-07


{'var': ['x4', 'x1'],
 'p_value': [0.0005762318164885003, 1.1052814195373175e-06]}

### Step 3

1. Suppose both $x4$ and $x1$ made it into the two-predictor stepwise model and remained there.

2. Now, fit each of the three-predictor models that include $x4$ and $x1$ as predictors — that is, regress $y$ on $x4$, $x1$, and $x2$, regress $y$ on $x4$, $x1$, and $x3$, ..., and regress $y$ on $x4$, $x1$, and $xp-1$.
3. Of those predictors whose t-test P-value is less than $\alpha_{\mathrm{E}}=0.15$, the third predictor put in the stepwise model is the predictor that has the smallest t-test P-value.
4. If no predictor has a t-test P-value less than $\alpha_{\mathrm{E}}=0.15$, stop. The model containing the two predictors obtained from the second step is your final model.

In [0]:
### Forward
p_value_final = 0.15
var_final = ''

for var in list_x:
  if var not in dic_stepwise['var']:
    list_temps = []
    for value in dic_stepwise['var']:
      list_temps.append(value)
    list_temps.append(var)
    
    x = df[list_temps].to_numpy()
    x = sm.add_constant(x)
    model = sm.OLS(y,x).fit()
    p_value = model.pvalues[2]
    print("p_value:" +  str(p_value))
    
    if p_value < p_value_final:
      p_value_final = p_value
      var_final =  var
print(var_final, p_value_final)

p_value:5.780763673513142e-07
p_value:0.0011163909896500272
x2 5.780763673513142e-07


5. But, instead that $x2$ was deemed the "best" third predictor and it is therefore entered into the stepwise model.
6. Now, since $x4$ and $x1$ were the first predictors in the model, step back and see if entering $x2$ into the stepwise model somehow affected the significance of the $x4$ and $x1$ predictors. That is, check the t-test P-values for testing $\beta_{1}=0$ and $\beta_{2}=0$. If the t-test P-value for either $\beta_{1}=0$or $\beta_{2}=0$ has become not significant — that is, the P-value is greater than $\alpha_{\mathrm{R}}=0.15$— remove the predictor from the stepwise model.

In [0]:
  ### backward
if var_final != '':
  
  list_temps = []
  for value in dic_stepwise['var']:
    list_temps.append(value)
  list_temps.append(var_final)
    
    
  x = df[list_temps].to_numpy()
  x = sm.add_constant(x)
  model = sm.OLS(y,x).fit()
  size_pvalue = len(model.pvalues)
  
  for i, value in enumerate(model.pvalues):
    if (i != size_pvalue) and (value > 0.15):
      print("p_value:" +  str(value) + " index " + str(i - 1))
      dic_stepwise['var'].pop(i-1)
      dic_stepwise['p_value'].pop(i - 1)
    elif (i == size_pvalue - 1) and (value  < 0.15):
      dic_stepwise['var'].append(var_final)
      dic_stepwise['p_value'].append(p_value_final)
      
dic_stepwise

0 0.0006753321641596413
1 0.20539543810168748
p_value:0.20539543810168748 index 0
2 5.780763673513142e-07
3 0.05168734897742293


### Repeat

Stopping the procedure. Continue the steps as described above until adding an additional predictor does not yield a t-test P-value below $\alpha_{\mathrm{E}}=0.15$

In [0]:
def stepwise_regression(df, X, y):
  y = df[y].to_numpy()
  dic_stepwise = {
    'var': [],
    'p_value': []
  }

  list_x = X

  p_value_final = 0.15
  var_final = ''
  worst_pvalue = 0
  #### Forward
  while worst_pvalue < 0.15:
  #changed = False
    p_value_final = 0.15
    var_final = ''
    for var in list_x:
    
      if var not in dic_stepwise['var']:
        list_temps = []
        for value in dic_stepwise['var']:
          list_temps.append(value)
        list_temps.append(var)
        x = df[list_temps].to_numpy()
        x = sm.add_constant(x)
        model = sm.OLS(y,x).fit()
    
        size_pvalue = len(model.pvalues)
        for i, value in enumerate(model.pvalues):
          if (i == size_pvalue - 1):
            p_value = model.pvalues[i]
      #print("p_value:" +  str(p_value))
    
        if p_value < p_value_final:
          p_value_final = p_value
          var_final =  var
        #changed = True
        
  #print(var_final)

  #### Backaward
    if var_final != '':
  
      list_temps = []
      for value in dic_stepwise['var']:
        list_temps.append(value)
      list_temps.append(var_final)
      print(list_temps)
      x = df[list_temps].to_numpy()
      x = sm.add_constant(x)
      model = sm.OLS(y,x).fit()
      size_pvalue = len(model.pvalues)
    
      for i, value in enumerate(model.pvalues):
        if (i != size_pvalue -1) and (value > 0.15):
          print("p_value:" +  str(value) + " index " + str(i - 1))
          dic_stepwise['var'].pop(i-1)
          dic_stepwise['p_value'].pop(i - 1)
        elif (i == size_pvalue - 1) and (value  < 0.15):
          dic_stepwise['var'].append(var_final)
          dic_stepwise['p_value'].append(p_value_final)
    
    worst_pvalue =  model.pvalues[1:].max()
    print(worst_pvalue)
    print(dic_stepwise['var'])
  
    
      
      


In [7]:
stepwise_regression(df, X = ['x1', 'x2', 'x4'], y = 'y')

['x4']
0.0005762318164885003
['x4']
['x4', 'x1']
1.1052814195373175e-06
['x4', 'x1']
['x4', 'x1', 'x2']
p_value:0.20539543810168748 index 0
0.20539543810168748
['x1', 'x2']


In [0]:
df[['x1', 'x2', 'x4']]

In [0]:
x = df[['x1', 'x2','x3', 'x4']]
x = sm.add_constant(x)
sm.OLS(y,x).fit().summary()

/Users/Thomas/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/Users/Thomas/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.974
Method:                 Least Squares   F-statistic:                     111.5
Date:                Mon, 29 Apr 2019   Prob (F-statistic):           4.76e-07
Time:                        19:59:00   Log-Likelihood:                -26.918
No. Observations:                  13   AIC:                             63.84
Df Residuals:                       8   BIC:                             66.66
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         62.4054     70.071      0.891      0.399     -99.179     223.989
x1             1.5511      0.745      2.083      0.071      -0.166       3.269
x2             0.5102      0.724      0.705      0.501      -1.159       2.179
x3             0.1019      0.755      0.135      0.896      -1.638       1.842
x4            -0.1441      0.709     -0.203      0.844      -1.779       1.491
==============================================================================
Omnibus:                        0.165   Durbin-Watson:                   2.053
Prob(Omnibus):                  0.921   Jarque-Bera (JB):                0.320
Skew:                           0.201   Prob(JB):                        0.852
Kurtosis:                       2.345   Cond. No.                     6.06e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [0]:
x = df[['x1', 'x2']]
x = sm.add_constant(x)
sm.OLS(y,x).fit().summary()
  

/Users/Thomas/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/Users/Thomas/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.974
Method:                 Least Squares   F-statistic:                     229.5
Date:                Mon, 29 Apr 2019   Prob (F-statistic):           4.41e-09
Time:                        19:56:34   Log-Likelihood:                -28.156
No. Observations:                  13   AIC:                             62.31
Df Residuals:                      10   BIC:                             64.01
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         52.5773      2.286     22.998      0.000      47.483      57.671
x1             1.4683      0.121     12.105      0.000       1.198       1.739
x2             0.6623      0.046     14.442      0.000       0.560       0.764
==============================================================================
Omnibus:                        1.509   Durbin-Watson:                   1.922
Prob(Omnibus):                  0.470   Jarque-Bera (JB):                1.104
Skew:                           0.503   Prob(JB):                        0.576
Kurtosis:                       1.987   Cond. No.                         175.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [0]:
x = df[['x1', 'x2', 'x3']]
x = sm.add_constant(x)
sm.OLS(y,x).fit().summary()
  

/Users/Thomas/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/Users/Thomas/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                     166.3
Date:                Mon, 29 Apr 2019   Prob (F-statistic):           3.37e-08
Time:                        19:56:04   Log-Likelihood:                -26.952
No. Observations:                  13   AIC:                             61.90
Df Residuals:                       9   BIC:                             64.16
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         48.1936      3.913     12.315      0.000      39.341      57.046
x1             1.6959      0.205      8.290      0.000       1.233       2.159
x2             0.6569      0.044     14.851      0.000       0.557       0.757
x3             0.2500      0.185      1.354      0.209      -0.168       0.668
==============================================================================
Omnibus:                        0.164   Durbin-Watson:                   2.110
Prob(Omnibus):                  0.921   Jarque-Bera (JB):                0.261
Skew:                           0.208   Prob(JB):                        0.878
Kurtosis:                       2.445   Cond. No.                         319.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [0]:
x = df[['x1', 'x2', 'x4']]
x = sm.add_constant(x)
sm.OLS(y,x).fit().summary()

/Users/Thomas/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/Users/Thomas/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                     166.8
Date:                Mon, 29 Apr 2019   Prob (F-statistic):           3.32e-08
Time:                        19:56:07   Log-Likelihood:                -26.933
No. Observations:                  13   AIC:                             61.87
Df Residuals:                       9   BIC:                             64.13
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         71.6483     14.142      5.066      0.001      39.656     103.641
x1             1.4519      0.117     12.410      0.000       1.187       1.717
x2             0.4161      0.186      2.242      0.052      -0.004       0.836
x4            -0.2365      0.173     -1.365      0.205      -0.629       0.155
==============================================================================
Omnibus:                        0.211   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.900   Jarque-Bera (JB):                0.378
Skew:                           0.202   Prob(JB):                        0.828
Kurtosis:                       2.270   Cond. No.                     1.27e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.27e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Boston dataset

In [8]:
headers = ['CRIM','ZN','INDUS','NOX','RM','AGE','DIS','TAX','PTRATIO','MEDV']
range_name = 'boston_train.csv!A2:J402'

df = service['sheet'].spreadsheets().values().get(
    spreadsheetId='1IwxJCod9CKHim14ltdHlJwnYZktVcaCAS5d0pjvgaTM',
    range=range_name).execute()
df = pd.DataFrame(df.get('values', []), columns=headers)
df = df.apply(pd.to_numeric, errors='ignore')
df.head()

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,MEDV
0,2.30040,0.0,19.58,0.605,6.319,96.1,2.1000,403,14.7,23.8
1,13.35980,0.0,18.10,0.693,5.887,94.7,1.7821,666,20.2,12.7
2,0.12744,0.0,6.91,0.448,6.770,2.9,5.7209,233,17.9,26.6
3,0.15876,0.0,10.81,0.413,5.961,17.5,5.2873,305,19.2,21.7
4,0.03768,80.0,1.52,0.404,7.274,38.3,7.3090,329,12.6,34.6


In [0]:
stepwise_regression(df, X = ['CRIM','ZN','INDUS','NOX','RM','AGE','DIS',
                             'TAX','PTRATIO'],
                    y = 'MEDV')